# Presets

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Thu_Jun_11_22:26:38_PDT_2020
Cuda compilation tools, release 11.0, V11.0.194
Build cuda_11.0_bu.TC445_37.28540450_0


In [2]:
!pip install --user tensorflow==2.3.0

In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
# Clone the tensorflow models repository if it doesn't already exist

import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2193, done.
remote: Counting objects: 100% (2193/2193), done.
remote: Compressing objects: 100% (1897/1897), done.
remote: Total 2193 (delta 525), reused 933 (delta 270), pack-reused 0
Receiving objects: 100% (2193/2193), 30.42 MiB | 9.15 MiB/s, done.
Resolving deltas: 100% (525/525), done.


In [5]:
# Install the Object Detection API
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --user .

/home/jupyter/sparky-rooms-master/tensorflow-model/models/research
Processing /home/jupyter/sparky-rooms-master/tensorflow-model/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1578115 sha256=b0168a4d814dad09b5bd87382db2cb9fdfaa3be508d6283d217127601574dba2
  Stored in directory: /tmp/pip-ephem-wheel-cache-it7h0t1i/wheels/3c/18/a8/29c11aadef72c361904b4654a518e8aa556dd860a5c8247277
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


# Dataset

Download the [MSCOCO](https://cocodataset.org/#home) dataset and focus only on person annotations and images since we want to train for person detection. 

In [ ]:
# Download coco dataset and prepare records
%cd ~/sparky-rooms-master/tensorflow-model/
# Move custom download script to corresponding location to execute
%mv download_and_preprocess_mscoco_person.sh models/research/object_detection/dataset_tools/
# Go to root of sources
%cd models/research/
!bash object_detection/dataset_tools/download_and_preprocess_mscoco_person.sh ~/data/mscoco

/home/jupyter/sparky-rooms-master/tensorflow-model
mv: cannot stat 'download_and_preprocess_mscoco_person.sh': No such file or directory
/home/jupyter/sparky-rooms-master/tensorflow-model/models/research
Skipping download of train2017.zip
train2017
Skipping unzipping of train2017.zip
Skipping download of val2017.zip
val2017
Skipping unzipping of val2017.zip
Skipping download of test2017.zip
test2017
Skipping unzipping of test2017.zip
Skipping download of annotations_trainval2017.zip
annotations_trainval2017
Unzipping annotations_trainval2017.zip
Skipping download of image_info_test2017.zip
image_info_test2017
Unzipping image_info_test2017.zip
2020-09-24 07:51:20.275517: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2020-09-24 07:51:20.275

# Training

Reference our configured pipeline and output of the training

In [ ]:
CONFIG="pipeline.config"
MODEL_OUT="models/ssd_mobilenet_v2_fpn_sparky"

In [ ]:
# Run training
%cd ~/sparky-rooms-master/tensorflow-model/
!python models/research/object_detection/model_main_tf2.py \
--model_dir=$MODEL_OUT \
--num_train_steps=10000 \
--sample_1_of_n_eval_examples=1 \
--pipeline_config_path=$CONFIG \
--alsologtostderr

# Test model

# Export model

In [38]:
RESULTS="models/results/saved_model"
MODEL_NO_QUANT_TFLITE="models/results/model_no_quant.tflite"
MODEL_MICRO="models/results/model.cc"

In [ ]:
# Export checkpoint to .pb
!python models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path=$CONFIG \
    --trained_checkpoint_dir=$MODEL_OUT \
    --output_directory=$RESULTS

In [36]:
import tensorflow as tf

# Convert the model to the TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model(RESULTS)
model_no_quant_tflite = converter.convert()

open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

516

In [40]:
# Convert to a C source file
!apt-get update && apt-get -qq install xxd
!xxd -i {MODEL_NO_QUANT_TFLITE} > {MODEL_MICRO}
REPLACE_TEXT = MODEL_NO_QUANT_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_MICRO}

Reading package lists... Done
W: chmod 0700 of directory /var/lib/apt/lists/partial failed - SetupAPTPartialDirectory (1: Operation not permitted)
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
